In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# read data
df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])
df_users = pd.read_csv("file\\ml-1m\\users.dat", sep="::", header=None, names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])

<ipython-input-4-66699cb8c506>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
<ipython-input-4-66699cb8c506>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])
<ipython-input-4-66699cb8c506>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as reg

In [5]:
df_ratings.shape, df_movies.shape, df_users.shape

((1000209, 4), (3883, 3), (6040, 5))

In [6]:
df_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
df_movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
df_users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
# merge data
df_ratings_movies = pd.merge(df_ratings, df_movies, on='MovieID')
df = pd.merge(df_ratings_movies, df_users, on="UserID")
df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [10]:
# 필요없는 열 삭제
data = df.copy()
data = data.drop(['Timestamp','Genres','Gender','Age','Occupation','Zip-code'], axis=1)
data.head()

,UserID,MovieID,Rating,Title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [12]:
# 가장 많이 본 영화 5개 추천
def recommend_movie(n):
    movie_rank = data['MovieID'].value_counts()[:n]
    recommend_movies = data.loc[movie_rank.index]
    recommend = recommend_movies['Title']
    return recommend

recommend_movie(5)

2858                Muppet Treasure Island (1996)
260                            Dragonheart (1996)
1196    Close Encounters of the Third Kind (1977)
1210                   Breakfast Club, The (1985)
480                          Shanghai Noon (2000)
Name: Title, dtype: object

In [73]:
# train, test set 분리
x = data.copy()
x_train, x_test = train_test_split(x, test_size=0.05)

# dataset에서 가장 인기 있는 영화 n개 추천
def popular_movie(dataset, n):
    movie_rank = dataset['MovieID'].value_counts()[:n]
    popular_movies = dataset.iloc[movie_rank.index]
    return popular_movies

# Hitrate@K 계산
def hitrate(K):
    raw_ranking = popular_movie(data, K)
    pred_ranking = popular_movie(x_test, K)
    return raw_ranking['MovieID'].isin(pred_ranking['MovieID']).value_counts(normalize=True)[True]

hitrate(100)

0.06

In [14]:
####### 수정 필요

# train, test set 분리
x = data.copy()
x_train, x_test = train_test_split(x, test_size=0.05)

# 영화의 순위 계산
def movie_rank(input_data, movie_id):
    movie_rank = input_data['MovieID'].value_counts()
    ranking = movie_rank.rank(ascending=False, method='min')
    return round(ranking[movie_id])

# 영화 순위 예측치를 계산하는 기본 모델
def pred_rank(movie_id):
    try:
        rank = movie_rank(x_train, movie_id)
    except:
        rank = 1853    # 영화 3706개이므로 중간값
    return rank

# 정확도 계산
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE 계산
def score(model):
    y_pred = np.array([model(movie) for movie in x_test['MovieID']])
    y_true = np.array([movie_rank(data, movie) for movie in x_test['MovieID']])
    return RMSE(y_true, y_pred)


score(pred_rank)

23.285703998755164

In [84]:
# 평점 높은 영화 5개 추천
def recommend_movie2(n):
    movie_sort = movie_mean.sort_values(ascending=False)[:n]
    recommend_movies = data.loc[movie_sort.index]
    recommendation = recommend_movies['Title']
    return recommendation

movie_mean = data.groupby(['MovieID'])['Rating'].mean()
recommend_movie2(5)

MovieID
3382                        Sneakers (1992)
3172                              Pi (1998)
3607                        Candyman (1992)
3656                       Junk Mail (1997)
3280    Teenage Mutant Ninja Turtles (1990)
Name: Title, dtype: object

In [85]:
# 정확도 계산
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []
for user in set(data.index):
    y_true = data.loc[user]['Rating']
    y_pred = movie_mean[data.loc[user]['MovieID']]
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy)
print(np.mean(rmse))

0.7787902185267609


In [91]:
# train, test set 분리
x = data.copy()
y = data['UserID']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.05,stratify=y)

# 정확도 계산
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE 계산
def score(model):
    id_pairs = zip(x_test['UserID'], x_test['MovieID'])
    y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
    y_true = np.array(x_test['Rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 FULL Matrix 구하기
rating_matrix = x_train.pivot(index='UserID', columns='MovieID', values='Rating')

# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

train_mean = x_train.groupby(['MovieID'])['Rating'].mean()
score(best_seller)

0.9765758608443047